In [1]:
import requests
import spotipy
from requests.auth import HTTPBasicAuth
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

In [2]:
from client import client_id
from client import client_secret

In [3]:
top_artist_df = pd.read_csv("top2018.csv")
top_artist_df.head()

,id,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,6DCZcSspjsKoFjzjrWoCd,God's Plan,Drake,0.754,0.449,7.0,-9.211,1.0,0.1090,0.0332,0.000083,0.552,0.357,77.169,198973.0,4.0
1,3ee8Jmje8o58CHK66QrVC,SAD!,XXXTENTACION,0.740,0.613,8.0,-4.880,1.0,0.1450,0.2580,0.003720,0.123,0.473,75.023,166606.0,4.0
2,0e7ipj03S05BNilyu5bRz,rockstar (feat. 21 Savage),Post Malone,0.587,0.535,5.0,-6.090,0.0,0.0898,0.1170,0.000066,0.131,0.140,159.847,218147.0,4.0
3,3swc6WTsr7rl9DqQKQA55,Psycho (feat. Ty Dolla $ign),Post Malone,0.739,0.559,8.0,-8.011,1.0,0.1170,0.5800,0.000000,0.112,0.439,140.124,221440.0,4.0
4,2G7V7zsVDxg1yRsu7Ew9R,In My Feelings,Drake,0.835,0.626,1.0,-5.833,1.0,0.1250,0.0589,0.000060,0.396,0.350,91.030,217925.0,4.0


In [4]:
token_url = "https://accounts.spotify.com/api/token"
headers = {'content-type': 'application/x-www-form-urlencoded'}

token_params = {"grant_type" : "client_credentials"}

token_json = requests.post(token_url, params = token_params, headers = headers, auth = (client_id, client_secret)).json()
token_json

{'access_token': 'BQCxq0wtPyfcNFhHAKXLfCQFu8Yh0iJgTAsJq0jGDRHwLk-xGDh9EsXx8kx-Jap89MxZlIzOB-HJPyIQ32o',
 'token_type': 'Bearer',
 'expires_in': 3600,
 'scope': ''}

In [15]:
token = token_json['access_token']

token_headers = {'authorization': f"Bearer {token}"}


artist_id = []
search_url = "https://api.spotify.com/v1/search"

print("----------------------\nRetrieving Artist ID\n----------------------")

for artist in top_artist_df["artists"]:
    search_param = {"q" : artist,
                   "type" : "artist",
                   "limit" : 1}
    
    try:
        artist_json = requests.get(search_url, params = search_param, headers = token_headers).json()
        artist_ids = artist_json["artists"]["items"][0]["id"]

        if artist_ids not in artist_id:
            artist_id.append(artist_ids)
            print(f"Found Artist ID | {artist_ids}")
            
        else:
            print("Artist ID in list already")
        
    except IndexError:
        print("Could Not Find Artist ID")






# track_url = "https://api.spotify.com/v1/tracks/"

# token_headers = {'authorization': f"Bearer {token}"}

# for track_id in top_artist_df["id"]:
#     pprint(requests.get(track_url + str(track_id), headers = token_headers).json())

----------------------
Retrieving Artist ID
----------------------
Found Artist ID | 3TVXtAsR1Inumwj472S9r4
Found Artist ID | 15UsOTVnJzReFVN1VCnxy4
Found Artist ID | 246dkjvS1zLTtiykXe5h60
Artist ID in list already
Artist ID in list already
Artist ID in list already
Found Artist ID | 4kYSro6naA4h99UJvo89HB
Found Artist ID | 7CajNmpbOovFoOoasH2HaY
Found Artist ID | 6M2wZ9GZgrQXHCFfjv46we
Found Artist ID | 64KEffDW9EtZ1y2vBYgq8T
Found Artist ID | 4nDoRrQiYLoBzwC5BhVJzF
Found Artist ID | 4MCBfE4596Uoi2O4DtmEMz
Artist ID in list already
Found Artist ID | 04gDigrS5kc9YWfZHwBETP
Found Artist ID | 2qxJFvFYMEDqd7ui6kSAcq
Found Artist ID | 2YZyLoL8N0Wb9xBt1NhZWg
Found Artist ID | 66CXWjxzNUsdJxJ2JdwvnR
Found Artist ID | 1SupJlEpv7RS2tPNRaHViT
Artist ID in list already
Found Artist ID | 4TEJudQY2pXxVHPE3gD2EU
Found Artist ID | 4WN5naL3ofxrVBgFpguzKo
Found Artist ID | 5hdhHgpxyniooUiQVaPxQ0
Found Artist ID | 4GvEc3ANtPPjt1ZJllr5Zl
Found Artist ID | 5Rl15oVamLq7FbSb0NNBNy
Artist ID in list alread

In [9]:
# artist_url = "https://api.spotify.com/v1/artists/"
# artist_id = track_json["album"]["artists"][0]['id']


# artist_json = requests.get(artist_url + artist_id, headers = token_headers).json()

In [ ]:
##EXAMPLE


#requests.get("https://api.spotify.com/v1/tracks/6DCZcSspjsKoFjzjrWoCdn", headers = token_headers).json()